# Section: Initialization

- setting values for global variables
- loading required packages
- reading files
- setting up logging

In [1]:
# This code is for supervised learning using previous state to predict current action
# Code from gym-flight to be reused as much as possible
import tensorflow as tf
import sys
import os
sys.path.append('../gym-flight/')
import gym_flight
from gym_flight.utils.numpy_util import Sparse3dArray
import numpy as np
import math
import pandas as pd
from multiclass import *
from full3d_util import get_last_ts_data, create_space_ts, get_range_df, get_action_discrete, get_env_action_aircraft, get_X_Y, fix_XY, unlist, bind, train_test_split

In [2]:
import logging
logging.basicConfig(
    filename='log.txt',
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
x_length = 100
y_length = 100
z_length = 100
ts_range = 40
time_dim = 1
state_dim = [x_length, y_length, z_length]
action_size = 36
learning_rate = 0.00025
flight_data_path = "../gym-flight/data/processed_jfk.csv"
dtype_dict = {"id": str, "ts": np.int16, "lat": np.float32, "lon": np.float32, "altitude": np.float32, "speed": np.float32, "x": np.int16, "y": np.int16, "z": np.int16, "is_landing": np.int8}
flight_data = pd.read_csv(flight_data_path, dtype = dtype_dict)

# Section on creating space for timestamp and aircraft-timestamp

In [3]:
os.path.isfile('ts_full3d_space.bin')

True

Multiprocessing the creation of space at unique timestamp and caching the output

In [4]:
import multiprocessing as mp
import pickle
splits = mp.cpu_count() - 1
if os.path.isfile('ts_full3d_space.bin'):
    space_X = pickle.load(open('ts_full3d_space.bin', 'rb'))
else:
    p = mp.Pool(processes = splits)
    # Arranging in reverse order to speed up computation
    rng = list(reversed(range(min(flight_data['ts']), max(flight_data['ts']) + 1)))
    split_rng = np.array_split(rng, splits)
    pool_results = p.map(get_range_df, split_rng)
    p.close()
    p.join()
    space_X = np.concatenate(pool_results, axis = 0)
    space_X = np.flip(space_X, axis = 0)
    pickle.dump(space_X, open("ts_full3d_space.bin", "wb"))


In [5]:
len(space_X)

1459

Calls functions to obtain the sparse array of environments for each aircraft-timestamp

In [6]:
if os.path.isfile('X_train_full3d_space_multiclass_action.bin'):
    X_train = pickle.load(open('X_train_full3d_space_multiclass_action.bin', 'rb'))
    Y_train = pickle.load(open('Y_train_full3d_space_multiclass_action.bin', 'rb'))
    X_test = pickle.load(open('X_test_full3d_space_multiclass_action.bin', 'rb'))
    Y_test = pickle.load(open('Y_test_full3d_space_multiclass_action.bin', 'rb'))
else:
    uniq_id = flight_data['id'].unique()
    #     uniq_id = uniq_id[:50]
    XY = get_X_Y(uniq_id, flight_data, x_length, y_length, z_length, space_X, discrete_action = True)
    X, Y = fix_XY(XY)
    X_train, Y_train, X_test, Y_test = train_test_split(X, Y)
    pickle.dump(X_train, open("X_train_full3d_space_multiclass_action.bin", "wb"))
    pickle.dump(Y_train, open("Y_train_full3d_space_multiclass_action.bin", "wb"))
    pickle.dump(X_test, open("X_test_full3d_space_multiclass_action.bin", "wb"))
    pickle.dump(Y_test, open("Y_test_full3d_space_multiclass_action.bin", "wb"))

train_ac_index = [len(X[0]) for X in X_train]
test_ac_index = [len(X[0]) for X in X_test]
X_train = unlist(unlist(X_train))
Y_train = bind(unlist(Y_train))
Y_train = Y_train.astype(np.int16)
X_test = unlist(unlist(X_test))
Y_test = bind(unlist(Y_test))
Y_test = Y_test.astype(np.int16)

# Section on CNN, prediction, training and evaluation

In [7]:
state_dim = [x_length, y_length, z_length]
cnn = CNN_multiclass(state_dim, time_dim, action_size, learning_rate)

Instructions for updating:
Use keras.layers.conv3d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


# Section on using the CNN class and data for training, prediction and evaluation

In [8]:
saver = tf.train.Saver()

# unique_ts = flight_data['ts'].unique()
# for i in range(len(unique_ts)):
#     train_X.append(create_space_ts(ts = unique_ts.iloc[i], flight_data = flight_data, x_length = x_length, y_length = y_length, z_length = z_length, ts_range = ts_range))

# Row binding all the arrays
# train_X = np.concatenate(train_X, axis = 0)
perc = 0
epochs = 10
batch_size = 128
num_batches = len(X_train)//batch_size


In [9]:
num_batches

4459

In [10]:
completed = 0
epoch_train_losses = []
epoch_test_losses = []
ended = False
epoch = 0
sess = tf.Session()
# Initialize variables
sess.run(tf.global_variables_initializer())
while (epoch < epochs) and not(ended):
    completed = int(epoch * 100/epochs)
    if completed >= perc:
        logging.info(str(perc) + " % completed")
        perc = int(epoch * 100/epochs)

    batch_completed = 0
    mini_batch = 1
    mini_batch_train_losses = []
    mini_batch_test_losses = []
    while batch_completed < (len(X_train) - batch_size):
        train_X = X_train[batch_completed:(batch_completed + batch_size)]
        train_Y = Y_train[batch_completed:(batch_completed + batch_size)]
        loss = train(sess, cnn, train_X, train_Y)
        # Runs out of memory while evaluating on the complete test set. Only batch_size used for evaluating
        # This step should be randomized
        test_loss = get_loss(sess, cnn, X_test[0:batch_size], Y_test[0:batch_size])
        print('Epoch: {}/{}: '.format(epoch+1, epochs), 'mini-batch {}/{}: '.format(mini_batch, num_batches), "Train loss: {} ".format(loss), "Test loss: {} ".format(test_loss))
        batch_completed += batch_size
        mini_batch += 1
        mini_batch_train_losses.append(loss)
        mini_batch_test_losses.append(test_loss)
    epoch_train_losses.append(sum(mini_batch_train_losses)/len(mini_batch_train_losses))
    epoch_test_losses.append(sum(mini_batch_test_losses)/len(mini_batch_test_losses))
    # Early stopping check (callback should be used instead):
    if (epoch > 0) and (epoch_test_losses[epoch] > epoch_test_losses[epoch - 1]):
        saver.save(sess, "supervised_CNN_full3d_multiclass")
        ended = True

    epoch += 1

Epoch: 1/10:  mini-batch 1/4459:  Train loss: 3.5838890075683594  Test loss: 3.581315279006958 
Epoch: 1/10:  mini-batch 2/4459:  Train loss: 3.5826973915100098  Test loss: 3.5760655403137207 
Epoch: 1/10:  mini-batch 3/4459:  Train loss: 3.584693670272827  Test loss: 3.575218677520752 
Epoch: 1/10:  mini-batch 4/4459:  Train loss: 3.5847012996673584  Test loss: 3.575793743133545 
Epoch: 1/10:  mini-batch 5/4459:  Train loss: 3.5747973918914795  Test loss: 3.574061870574951 
Epoch: 1/10:  mini-batch 6/4459:  Train loss: 3.5790867805480957  Test loss: 3.570615530014038 
Epoch: 1/10:  mini-batch 7/4459:  Train loss: 3.5394880771636963  Test loss: 3.564418315887451 
Epoch: 1/10:  mini-batch 8/4459:  Train loss: 3.557560920715332  Test loss: 3.555276393890381 
Epoch: 1/10:  mini-batch 9/4459:  Train loss: 3.5063180923461914  Test loss: 3.5420680046081543 
Epoch: 1/10:  mini-batch 10/4459:  Train loss: 3.424839496612549  Test loss: 3.52697491645813 
Epoch: 1/10:  mini-batch 11/4459:  Train 

Epoch: 1/10:  mini-batch 86/4459:  Train loss: 2.7636795043945312  Test loss: 4.28445291519165 
Epoch: 1/10:  mini-batch 87/4459:  Train loss: 2.0503787994384766  Test loss: 4.3655853271484375 
Epoch: 1/10:  mini-batch 88/4459:  Train loss: 2.361149787902832  Test loss: 4.476391792297363 
Epoch: 1/10:  mini-batch 89/4459:  Train loss: 2.503573179244995  Test loss: 4.589815139770508 
Epoch: 1/10:  mini-batch 90/4459:  Train loss: 2.6895573139190674  Test loss: 4.667757987976074 
Epoch: 1/10:  mini-batch 91/4459:  Train loss: 2.7669246196746826  Test loss: 4.691670894622803 
Epoch: 1/10:  mini-batch 92/4459:  Train loss: 2.1918773651123047  Test loss: 4.73465633392334 
Epoch: 1/10:  mini-batch 93/4459:  Train loss: 2.2633485794067383  Test loss: 4.794447898864746 
Epoch: 1/10:  mini-batch 94/4459:  Train loss: 3.5106050968170166  Test loss: 4.693515300750732 
Epoch: 1/10:  mini-batch 95/4459:  Train loss: 2.367588996887207  Test loss: 4.6100993156433105 
Epoch: 1/10:  mini-batch 96/4459:

Epoch: 1/10:  mini-batch 170/4459:  Train loss: 2.57767391204834  Test loss: 4.942663192749023 
Epoch: 1/10:  mini-batch 171/4459:  Train loss: 2.6753015518188477  Test loss: 5.1206440925598145 
Epoch: 1/10:  mini-batch 172/4459:  Train loss: 2.528700828552246  Test loss: 5.270549297332764 
Epoch: 1/10:  mini-batch 173/4459:  Train loss: 2.5182907581329346  Test loss: 5.384564399719238 
Epoch: 1/10:  mini-batch 174/4459:  Train loss: 2.585728168487549  Test loss: 5.441375732421875 
Epoch: 1/10:  mini-batch 175/4459:  Train loss: 2.5374345779418945  Test loss: 5.448753356933594 
Epoch: 1/10:  mini-batch 176/4459:  Train loss: 2.5578830242156982  Test loss: 5.398690700531006 
Epoch: 1/10:  mini-batch 177/4459:  Train loss: 2.4982380867004395  Test loss: 5.318838119506836 
Epoch: 1/10:  mini-batch 178/4459:  Train loss: 2.7446742057800293  Test loss: 5.187656879425049 
Epoch: 1/10:  mini-batch 179/4459:  Train loss: 2.4412002563476562  Test loss: 5.058216571807861 
Epoch: 1/10:  mini-batc

Epoch: 1/10:  mini-batch 254/4459:  Train loss: 3.6145148277282715  Test loss: 3.5773234367370605 
Epoch: 1/10:  mini-batch 255/4459:  Train loss: 3.4526467323303223  Test loss: 3.5763309001922607 
Epoch: 1/10:  mini-batch 256/4459:  Train loss: 3.5328731536865234  Test loss: 3.575489044189453 
Epoch: 1/10:  mini-batch 257/4459:  Train loss: 3.6433019638061523  Test loss: 3.574758529663086 
Epoch: 1/10:  mini-batch 258/4459:  Train loss: 3.606125831604004  Test loss: 3.574162006378174 
Epoch: 1/10:  mini-batch 259/4459:  Train loss: 3.534118175506592  Test loss: 3.573662757873535 
Epoch: 1/10:  mini-batch 260/4459:  Train loss: 3.52670955657959  Test loss: 3.573364496231079 
Epoch: 1/10:  mini-batch 261/4459:  Train loss: 3.479160785675049  Test loss: 3.5731165409088135 
Epoch: 1/10:  mini-batch 262/4459:  Train loss: 3.6229052543640137  Test loss: 3.5727498531341553 
Epoch: 1/10:  mini-batch 263/4459:  Train loss: 3.58607816696167  Test loss: 3.572467088699341 
Epoch: 1/10:  mini-batc

Epoch: 1/10:  mini-batch 338/4459:  Train loss: 3.601224422454834  Test loss: 3.530951976776123 
Epoch: 1/10:  mini-batch 339/4459:  Train loss: 3.4151647090911865  Test loss: 3.527974843978882 
Epoch: 1/10:  mini-batch 340/4459:  Train loss: 3.626239776611328  Test loss: 3.524399995803833 
Epoch: 1/10:  mini-batch 341/4459:  Train loss: 3.14350962638855  Test loss: 3.5218019485473633 
Epoch: 1/10:  mini-batch 342/4459:  Train loss: 3.4967689514160156  Test loss: 3.519287109375 
Epoch: 1/10:  mini-batch 343/4459:  Train loss: 3.6646931171417236  Test loss: 3.516209602355957 
Epoch: 1/10:  mini-batch 344/4459:  Train loss: 3.668013095855713  Test loss: 3.5131802558898926 
Epoch: 1/10:  mini-batch 345/4459:  Train loss: 3.453693389892578  Test loss: 3.5103988647460938 
Epoch: 1/10:  mini-batch 346/4459:  Train loss: 3.188122272491455  Test loss: 3.508371353149414 
Epoch: 1/10:  mini-batch 347/4459:  Train loss: 3.395141124725342  Test loss: 3.5060503482818604 
Epoch: 1/10:  mini-batch 34

...

Epoch: 2/10:  mini-batch 1774/4459:  Train loss: 3.1680054664611816  Test loss: 3.3360719680786133 
Epoch: 2/10:  mini-batch 1775/4459:  Train loss: 3.143526315689087  Test loss: 3.3361878395080566 
Epoch: 2/10:  mini-batch 1776/4459:  Train loss: 3.3655147552490234  Test loss: 3.3365845680236816 
Epoch: 2/10:  mini-batch 1777/4459:  Train loss: 3.2314915657043457  Test loss: 3.3369669914245605 
Epoch: 2/10:  mini-batch 1778/4459:  Train loss: 2.9342403411865234  Test loss: 3.3372578620910645 
Epoch: 2/10:  mini-batch 1779/4459:  Train loss: 2.9234821796417236  Test loss: 3.3376121520996094 
Epoch: 2/10:  mini-batch 1780/4459:  Train loss: 2.9961752891540527  Test loss: 3.3380417823791504 
Epoch: 2/10:  mini-batch 1781/4459:  Train loss: 3.6323533058166504  Test loss: 3.3386149406433105 
Epoch: 2/10:  mini-batch 1782/4459:  Train loss: 3.3401575088500977  Test loss: 3.339142322540283 
Epoch: 2/10:  mini-batch 1783/4459:  Train loss: 3.430330276489258  Test loss: 3.3401119709014893 
Epo

Epoch: 2/10:  mini-batch 1857/4459:  Train loss: 3.3471245765686035  Test loss: 3.369520664215088 
Epoch: 2/10:  mini-batch 1858/4459:  Train loss: 3.4733433723449707  Test loss: 3.369843006134033 
Epoch: 2/10:  mini-batch 1859/4459:  Train loss: 3.4076244831085205  Test loss: 3.370347738265991 
Epoch: 2/10:  mini-batch 1860/4459:  Train loss: 3.4754018783569336  Test loss: 3.370495319366455 
Epoch: 2/10:  mini-batch 1861/4459:  Train loss: 3.2805774211883545  Test loss: 3.370849847793579 
Epoch: 2/10:  mini-batch 1862/4459:  Train loss: 3.3091306686401367  Test loss: 3.371244430541992 
Epoch: 2/10:  mini-batch 1863/4459:  Train loss: 3.337725877761841  Test loss: 3.3713560104370117 
Epoch: 2/10:  mini-batch 1864/4459:  Train loss: 3.286978006362915  Test loss: 3.371645450592041 
Epoch: 2/10:  mini-batch 1865/4459:  Train loss: 3.0136477947235107  Test loss: 3.3718104362487793 
Epoch: 2/10:  mini-batch 1866/4459:  Train loss: 3.181356430053711  Test loss: 3.3719019889831543 
Epoch: 2/1

Epoch: 2/10:  mini-batch 1940/4459:  Train loss: 3.14365816116333  Test loss: 3.3993797302246094 
Epoch: 2/10:  mini-batch 1941/4459:  Train loss: 3.197895050048828  Test loss: 3.399432420730591 
Epoch: 2/10:  mini-batch 1942/4459:  Train loss: 3.396986246109009  Test loss: 3.39955997467041 
Epoch: 2/10:  mini-batch 1943/4459:  Train loss: 3.2438645362854004  Test loss: 3.4007959365844727 
Epoch: 2/10:  mini-batch 1944/4459:  Train loss: 3.3907294273376465  Test loss: 3.403146266937256 
Epoch: 2/10:  mini-batch 1945/4459:  Train loss: 3.369664430618286  Test loss: 3.4045910835266113 
Epoch: 2/10:  mini-batch 1946/4459:  Train loss: 3.1921966075897217  Test loss: 3.4064855575561523 
Epoch: 2/10:  mini-batch 1947/4459:  Train loss: 3.420464515686035  Test loss: 3.406790256500244 
Epoch: 2/10:  mini-batch 1948/4459:  Train loss: 3.3309597969055176  Test loss: 3.407956123352051 
Epoch: 2/10:  mini-batch 1949/4459:  Train loss: 3.4746289253234863  Test loss: 3.407832145690918 
Epoch: 2/10: 

Epoch: 2/10:  mini-batch 2023/4459:  Train loss: 3.4150609970092773  Test loss: 3.4588847160339355 
Epoch: 2/10:  mini-batch 2024/4459:  Train loss: 3.5448923110961914  Test loss: 3.458601951599121 
Epoch: 2/10:  mini-batch 2025/4459:  Train loss: 3.265249729156494  Test loss: 3.458801746368408 
Epoch: 2/10:  mini-batch 2026/4459:  Train loss: 3.7945070266723633  Test loss: 3.45780611038208 
Epoch: 2/10:  mini-batch 2027/4459:  Train loss: 3.521066188812256  Test loss: 3.456003427505493 
Epoch: 2/10:  mini-batch 2028/4459:  Train loss: 3.3252363204956055  Test loss: 3.4550416469573975 
Epoch: 2/10:  mini-batch 2029/4459:  Train loss: 3.401651382446289  Test loss: 3.4544293880462646 
Epoch: 2/10:  mini-batch 2030/4459:  Train loss: 3.1628284454345703  Test loss: 3.4543709754943848 
Epoch: 2/10:  mini-batch 2031/4459:  Train loss: 3.3156471252441406  Test loss: 3.4534733295440674 
Epoch: 2/10:  mini-batch 2032/4459:  Train loss: 3.443488359451294  Test loss: 3.4514565467834473 
Epoch: 2/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Loss for random guess

In [11]:
math.log(36)

3.58351893845611

In [12]:
tbl = {}
for y in Y_train:
    try:
        tbl[y] += 1
    except:
        tbl[y] = 1

tbl

{1: 30471,
 4: 46873,
 3: 37407,
 19: 26342,
 35: 11897,
 28: 9840,
 22: 36674,
 34: 8350,
 33: 2056,
 32: 10720,
 31: 13937,
 25: 33460,
 13: 16128,
 10: 10893,
 16: 8483,
 12: 20853,
 14: 4373,
 7: 33647,
 9: 19946,
 15: 5855,
 11: 1845,
 23: 23448,
 29: 4070,
 5: 6840,
 0: 22925,
 6: 7982,
 21: 14095,
 18: 23582,
 24: 6591,
 2: 5448,
 26: 22685,
 17: 7270,
 20: 6991,
 27: 7433,
 30: 6368,
 8: 15015}

# Section on testing the prediction the whole history of the first test aircraft

In [ ]:
test_aircraft_data = X_test[0:test_ac_index[0]]
test_class_predictions = get_predicted_class(sess, cnn, X_test)


# Conclusion

Experiment failed for 2 reasons:

- if current location of aircraft is coded as -1, then fraction of windows for which the convolution picks the controlled aircraft is quite small
- convolution uses shared weights between 2 layers. These weights are independent of the location of the controlled aircraft